

# Pagerank to rank teams

The goal of this notebook is to rank teams with the pagerank algorithm and compare that with the Ridge linear regression method


In [49]:

import os
import random

import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns

from tqdm import tqdm

DATA_PATH = '../../data'


['id',
 'uid',
 'date',
 'attendance',
 'timeValid',
 'neutralSite',
 'conferenceCompetition',
 'recent',
 'competitors',
 'notes',
 'tournamentId',
 'startDate',
 'venue.id',
 'venue.fullName',
 'venue.address.city',
 'venue.address.state',
 'venue.capacity',
 'venue.indoor',
 'status.clock',
 'status.displayClock',
 'status.period',
 'status.type.id',
 'status.type.name',
 'status.type.state',
 'status.type.completed',
 'status.type.description',
 'status.type.detail',
 'status.type.shortDetail',
 'format.regulation.periods',
 'home.id',
 'home.uid',
 'home.location',
 'home.name',
 'home.abbreviation',
 'home.displayName',
 'home.shortDisplayName',
 'home.color',
 'home.isActive',
 'home.venue.id',
 'home.logo',
 'home.conferenceId',
 'away.id',
 'away.uid',
 'away.location',
 'away.name',
 'away.abbreviation',
 'away.displayName',
 'away.shortDisplayName',
 'away.color',
 'away.alternateColor',
 'away.isActive',
 'away.venue.id',
 'away.logo',
 'away.conferenceId',
 'home.alternate

In [70]:

from sportsdataverse.mbb import espn_mbb_pbp, load_mbb_schedule,espn_mbb_calendar,espn_mbb_schedule

schedule = espn_mbb_schedule(dates=2019, groups =100)
schedule['date'] = pd.to_datetime(schedule['date'].copy())
schedule = schedule.sort_values(by=['date'])

schedule['game_id']
# load_mbb_schedule()



,id,uid,date,attendance,timeValid,neutralSite,conferenceCompetition,recent,competitors,notes,...,away.displayName,away.shortDisplayName,away.color,away.alternateColor,away.isActive,away.venue.id,away.logo,away.conferenceId,home.alternateColor,status.type.altDetail
0,401123378,s:40~l:41~e:401123378~c:401123378,2019-03-19 22:40:00+00:00,0,True,True,False,False,"[{'id': '2504', 'uid': 's:40~l:41~t:2504', 'ty...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,Fairleigh Dickinson Knights,Fair Dickinson,28334a,ffffff,True,2076,https://a.espncdn.com/i/teamlogos/ncaa/500/161...,19,NaN,NaN
0,401123380,s:40~l:41~e:401123380~c:401123380,2019-03-20 01:10:00+00:00,11784,True,True,False,False,"[{'id': '2057', 'uid': 's:40~l:41~t:2057', 'ty...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,Temple Owls,Temple,A80532,a7a9ac,True,2011,https://a.espncdn.com/i/teamlogos/ncaa/500/218...,62,c9262d,NaN
0,401123379,s:40~l:41~e:401123379~c:401123379,2019-03-20 22:40:00+00:00,0,True,True,False,False,"[{'id': '2449', 'uid': 's:40~l:41~t:2449', 'ty...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,North Carolina Central Eagles,NC Central,880023,c2c3c0,True,2081,https://a.espncdn.com/i/teamlogos/ncaa/500/242...,16,ffffff,NaN
0,401123377,s:40~l:41~e:401123377~c:401123377,2019-03-21 01:17:00+00:00,11827,True,True,False,False,"[{'id': '9', 'uid': 's:40~l:41~t:9', 'type': '...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,St. John's Red Storm,St. John's,d10000,101010,True,2019,https://a.espncdn.com/i/teamlogos/ncaa/500/259...,4,f1f2f3,NaN
0,401123436,s:40~l:41~e:401123436~c:401123436,2019-03-21 16:15:00+00:00,0,True,True,False,False,"[{'id': '97', 'uid': 's:40~l:41~t:97', 'type':...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,Minnesota Golden Gophers,Minnesota,981a31,981a31,True,2094,https://a.espncdn.com/i/teamlogos/ncaa/500/135...,7,cccccc,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,401123390,s:40~l:41~e:401123390~c:401123390,2019-03-31 18:20:00+00:00,17174,True,True,False,False,"[{'id': '96', 'uid': 's:40~l:41~t:96', 'type':...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,Auburn Tigers,Auburn,03244d,f1f2f3,True,1953,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,23,ffffff,OT
0,401123387,s:40~l:41~e:401123387~c:401123387,2019-03-31 21:05:00+00:00,20125,True,True,False,False,"[{'id': '150', 'uid': 's:40~l:41~t:150', 'type...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,Michigan State Spartans,Michigan State,18453B,ffffff,True,1987,https://a.espncdn.com/i/teamlogos/ncaa/500/127...,7,f1f2f3,NaN
0,401123375,s:40~l:41~e:401123375~c:401123375,2019-04-06 22:09:00+00:00,0,True,True,False,False,"[{'id': '258', 'uid': 's:40~l:41~t:258', 'type...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,Auburn Tigers,Auburn,03244d,f1f2f3,True,1953,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,23,ffffff,NaN
0,401123376,s:40~l:41~e:401123376~c:401123376,2019-04-07 00:49:00+00:00,72711,True,True,False,False,"[{'id': '127', 'uid': 's:40~l:41~t:127', 'type...","[{'type': 'event', 'headline': 'MEN'S BASKETBA...",...,Texas Tech Red Raiders,Texas Tech,C80025,231f20,True,2047,https://a.espncdn.com/i/teamlogos/ncaa/500/264...,8,ffffff,NaN


In [50]:


# list(schedule)
# schedule.loc[schedule['home.name'].str.contains("Cavaliers")|schedule['away.name'].str.contains("Cavaliers")]
# # schedule['home.name']

# from sportsdataverse.mbb import espn_mbb_pbp, load_mbb_schedule

# season = 2019
# schedule = load_mbb_schedule([season])
# schedule['date'] = pd.to_datetime(schedule['date'])
# schedule = schedule.sort_values(by='date')
# schedule.tail(5)

# espn_mbb_pbp(401123374)
# 401123374
# espn_mbb_pbp(401123374)


In [19]:

def load_season_schedule(year):
    return pd.read_csv(os.path.join(DATA_PATH, f'ESPN/schedules/by_season/{year}.csv'))

schedule = load_season_schedule(2019)


In [22]:
schedule['datetime'] = pd.to_datetime(schedule['date'].copy())
schedule.sort_values(by='datetime')

,id,uid,date,attendance,timeValid,neutralSite,conferenceCompetition,recent,notes,startDate,...,season,groups.id,groups.name,groups.shortName,groups.isConference,tournamentId,PBP,team_box,player_box,datetime
5956,401089873,s:40~l:41~e:401089873~c:401089873,2018-11-06T17:30Z,1042,True,False,False,False,[],2018-11-06T17:30Z,...,2019,NaN,NaN,NaN,NaN,NaN,True,True,True,2018-11-06 17:30:00+00:00
5955,401089358,s:40~l:41~e:401089358~c:401089358,2018-11-06T21:00Z,1852,True,False,False,False,[],2018-11-06T21:00Z,...,2019,NaN,NaN,NaN,NaN,NaN,False,True,True,2018-11-06 21:00:00+00:00
5954,401089177,s:40~l:41~e:401089177~c:401089177,2018-11-06T23:00Z,1193,True,False,False,False,[],2018-11-06T23:00Z,...,2019,NaN,NaN,NaN,NaN,NaN,True,True,True,2018-11-06 23:00:00+00:00
5953,401090140,s:40~l:41~e:401090140~c:401090140,2018-11-06T23:05Z,1255,True,False,False,False,[],2018-11-06T23:05Z,...,2019,NaN,NaN,NaN,NaN,NaN,True,True,True,2018-11-06 23:05:00+00:00
5951,401083308,s:40~l:41~e:401083308~c:401083308,2018-11-06T23:30Z,8021,True,False,False,False,"[{'type': 'event', 'headline': 'HALL OF FAME T...",2018-11-06T23:30Z,...,2019,NaN,NaN,NaN,NaN,NaN,True,True,True,2018-11-06 23:30:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,401123698,s:40~l:41~e:401123698~c:401123698,2019-03-28T01:00Z,3982,True,False,False,False,"[{'type': 'event', 'headline': 'NIT - QUARTERF...",2019-03-28T01:00Z,...,2019,NaN,NaN,NaN,NaN,21.0,True,True,True,2019-03-28 01:00:00+00:00
3,401123699,s:40~l:41~e:401123699~c:401123699,2019-03-28T01:00Z,5500,True,False,False,False,"[{'type': 'event', 'headline': 'NIT - QUARTERF...",2019-03-28T01:00Z,...,2019,NaN,NaN,NaN,NaN,21.0,True,True,True,2019-03-28 01:00:00+00:00
2,401123693,s:40~l:41~e:401123693~c:401123693,2019-04-02T23:00Z,0,True,True,False,False,"[{'type': 'event', 'headline': 'NIT - SEMIFINA...",2019-04-02T23:00Z,...,2019,NaN,NaN,NaN,NaN,21.0,True,True,True,2019-04-02 23:00:00+00:00
1,401123694,s:40~l:41~e:401123694~c:401123694,2019-04-03T01:32Z,4599,True,True,False,False,"[{'type': 'event', 'headline': 'NIT - SEMIFINA...",2019-04-03T01:32Z,...,2019,NaN,NaN,NaN,NaN,21.0,True,True,True,2019-04-03 01:32:00+00:00


In [12]:

def load_season_team_boxscore(year):
    return pd.read_csv(os.path.join(DATA_PATH, f'ESPN/team_boxscores/{year}.csv'))

data = load_season_team_boxscore(2019)


In [16]:

data['datetime'] = pd.to_datetime(data['game_date'])
data.sort_values(by=['datetime']).tail(20)


,team_short_display_name,team_uid,team_alternate_color,team_color,team_display_name,team_name,team_logo,team_location,team_id,team_abbreviation,...,home_away,opponent_id,opponent_name,opponent_mascot,opponent_abbrev,game_id,season,season_type,game_date,datetime
11834,Xavier,s:40~l:41~t:2752,9ea2a2,002144,Xavier Musketeers,Musketeers,https://a.espncdn.com/i/teamlogos/ncaa/500/275...,Xavier,2752,XAV,...,HOME,251,Texas,Longhorns,TEX,401123707,2019,3,2019-03-24,2019-03-24
11832,Harvard,s:40~l:41~t:108,dbdbdb,990000,Harvard Crimson,Crimson,https://a.espncdn.com/i/teamlogos/ncaa/500/108...,Harvard,108,HARV,...,HOME,152,NC State,Wolfpack,NCST,401123706,2019,3,2019-03-24,2019-03-24
11831,TCU,s:40~l:41~t:2628,f1f2f3,3C377D,TCU Horned Frogs,Horned Frogs,https://a.espncdn.com/i/teamlogos/ncaa/500/262...,TCU,2628,TCU,...,AWAY,158,Nebraska,Cornhuskers,NEB,401123705,2019,3,2019-03-25,2019-03-25
11830,Nebraska,s:40~l:41~t:158,f5f1e7,F20017,Nebraska Cornhuskers,Cornhuskers,https://a.espncdn.com/i/teamlogos/ncaa/500/158...,Nebraska,158,NEB,...,HOME,2628,TCU,Horned Frogs,TCU,401123705,2019,3,2019-03-25,2019-03-25
11829,Colorado,s:40~l:41~t:38,ffd200,d1c57e,Colorado Buffaloes,Buffaloes,https://a.espncdn.com/i/teamlogos/ncaa/500/38.png,Colorado,38,COLO,...,AWAY,2450,Norfolk State,Spartans,NORF,401123702,2019,3,2019-03-26,2019-03-26
11828,Norfolk State,s:40~l:41~t:2450,NaN,046546,Norfolk State Spartans,Spartans,https://a.espncdn.com/i/teamlogos/ncaa/500/245...,Norfolk State,2450,NORF,...,HOME,38,Colorado,Buffaloes,COLO,401123702,2019,3,2019-03-26,2019-03-26
11823,Indiana,s:40~l:41~t:84,eeedeb,7D110C,Indiana Hoosiers,Hoosiers,https://a.espncdn.com/i/teamlogos/ncaa/500/84.png,Indiana,84,IU,...,AWAY,2724,Wichita State,Shockers,WICH,401123697,2019,3,2019-03-26,2019-03-26
11822,Wichita State,s:40~l:41~t:2724,ffc41e,0d0a03,Wichita State Shockers,Shockers,https://a.espncdn.com/i/teamlogos/ncaa/500/272...,Wichita State,2724,WICH,...,HOME,84,Indiana,Hoosiers,IU,401123697,2019,3,2019-03-26,2019-03-26
11821,TCU,s:40~l:41~t:2628,f1f2f3,3C377D,TCU Horned Frogs,Horned Frogs,https://a.espncdn.com/i/teamlogos/ncaa/500/262...,TCU,2628,TCU,...,AWAY,156,Creighton,Bluejays,CREI,401123696,2019,3,2019-03-27,2019-03-27
11820,Creighton,s:40~l:41~t:156,00235d,13299e,Creighton Bluejays,Bluejays,https://a.espncdn.com/i/teamlogos/ncaa/500/156...,Creighton,156,CREI,...,HOME,2628,TCU,Horned Frogs,TCU,401123696,2019,3,2019-03-27,2019-03-27
